<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***This is the Hybrid Research Agent. It combines both capabilities into a single, intelligent workflow.***

***The script will ask you at the beginning:***

*"Do you want to enter a TOPIC or upload a PDF?" and then execute the appropriate research strategy.*

**The Hybrid Workflow**

**Input Handler:**

Checks if you provided a text string or a file.

**Dynamic Planner:**

**If Topic:**

Generates exploration questions (Breadth).

**If PDF:**

Generates validation/fact-checking questions (Depth).

**Researcher:**

Searches Tavily (works the same for both).

**Contextual Synthesizer:**

Adapts the final report style based on whether it's a fresh topic or a paper review.


In [11]:
!pip install -q -U google-generativeai tavily-python pypdf

In [12]:
import os
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata
from google.colab import files
import pypdf
import io

# 1. Setup API Keys
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("Error: Please set your API keys in the Colab Secrets tab.")

# 2. Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# 3. Configure Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

print("Setup complete.")

Setup complete.


In [14]:
def get_pdf_text(uploaded_file):
    """Extracts text from a PDF binary."""
    pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file))
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() + "\n"
    return text

def generate_plan(source_text, mode="topic"):
    """
    Generates search queries based on the mode.
    Mode 'topic': Breaks down a general topic.
    Mode 'pdf': Finds gaps/validation points in a paper.
    """
    print(f"🤔 Generating research plan for mode: {mode}...")

    if mode == "topic":
        prompt = f"""
        You are a research planner. Break down this topic into 3 distinct, searchable questions.
        Topic: {source_text}
        Output: Return ONLY 3 questions separated by newlines. No numbering.
        """
    else: # mode == pdf
        prompt = f"""
        You are a Senior Researcher. Read the following paper excerpt and identify 3 questions
        to validate the claims or find the latest state-of-the-art comparisons.

        Paper Excerpt: {source_text[:15000]}

        Output: Return ONLY 3 search queries separated by newlines. No numbering.
        """

    response = model.generate_content(prompt)
    return [q.strip() for q in response.text.strip().split('\n') if q.strip()]

def perform_research(questions):
    """Executes the search for the generated questions."""
    print(f"🔎 Searching the web...")
    results = []
    for q in questions:
        print(f"   - Searching: {q}")
        try:
            # Using advanced search depth for better quality
            res = tavily.search(query=q, search_depth="advanced", max_results=2)
            content = "\n".join([r['content'] for r in res['results']])
            results.append(f"Query: {q}\nFindings: {content}")
        except Exception as e:
            print(f"Error searching {q}: {e}")
    return "\n\n".join(results)

def synthesize_report(source_text, research_data, mode="topic"):
    """Summarizes everything into a final report."""
    print(f"📝 Synthesizing final report...")

    if mode == "topic":
        prompt = f"""
        Write a comprehensive research summary on the topic below, incorporating the search findings.

        User Topic: {source_text}
        Search Data: {research_data}

        Format: Markdown with headers.
        """
    else:
        prompt = f"""
        Create a "Deep Dive" analysis of the provided research paper using the external search data.

        1. Summarize the Paper's core contribution.
        2. Compare it with the "External Validation" findings (State-of-the-art, updates, or contradictions).
        3. Provide a final conclusion on the paper's relevance today.

        Paper Text (Context): {source_text[:15000]}
        External Validation Data: {research_data}

        Format: Markdown with headers.
        """

    response = model.generate_content(prompt)

    if not response.candidates:
        feedback = response.prompt_feedback
        error_message = f"Model did not return any candidates. Finish reason: {feedback.finish_reason}. Safety ratings: {feedback.safety_ratings}"
        print(f"Error: {error_message}")
        return f"**Error generating report:** {error_message}"
    try:
        return response.text
    except ValueError as e:
        feedback = response.prompt_feedback
        error_message = f"Failed to access response.text: {e}. Finish reason: {feedback.finish_reason}. Safety ratings: {feedback.safety_ratings}"
        print(f"Error: {error_message}")
        return f"**Error generating report:** {error_message}"

In [18]:
from IPython.display import Markdown, display

print("🤖 RESEARCH AGENT STARTED")
print("Select Mode:")
print("1. Research a Topic (Enter text)")
print("2. Analyze a Paper (Upload PDF)")

choice = input("Enter 1 or 2: ")

source_text = ""
mode = ""

if choice == '1':
    mode = "topic"
    source_text = input("\nEnter your research topic: ")
    if not source_text:
        print("❌ Topic cannot be empty.")
        mode = None

elif choice == '2':
    mode = "pdf"
    print("\n⬆️ Please upload your PDF file now:")
    uploaded = files.upload()
    if uploaded:
        filename = next(iter(uploaded))
        print(f"Processing {filename}...")
        source_text = get_pdf_text(uploaded[filename])
    else:
        print("❌ No file uploaded.")
        mode = None
else:
    print("❌ Invalid choice.")

# Run the workflow if inputs are valid
if mode and source_text:
    print("-" * 50)

    # Fix: Re-initialize the model to a commonly available one
    # The 'gemini-1.5-flash' model might not be available or supported for generateContent in your region/configuration.
    # Changing it to 'gemini-flash-latest' for broader compatibility based on the list of available models.
    model = genai.GenerativeModel('gemini-flash-latest')

    # Step 1: Plan
    queries = generate_plan(source_text, mode=mode)
    print(f"📋 Plan: {queries}")

    # Step 2: Act
    web_data = perform_research(queries)

    # Step 3: Synthesize
    final_output = synthesize_report(source_text, web_data, mode=mode)

    print("\n" + "="*20 + " FINAL REPORT " + "="*20 + "\n")
    display(Markdown(final_output))


🤖 RESEARCH AGENT STARTED
Select Mode:
1. Research a Topic (Enter text)
2. Analyze a Paper (Upload PDF)
Enter 1 or 2: 1

Enter your research topic: Ocular toxoplasmosis
--------------------------------------------------
🤔 Generating research plan for mode: topic...
📋 Plan: ['What are the current international consensus guidelines regarding the optimal duration and combination of antimicrobial therapy for active ocular toxoplasmosis?', 'How do genotypic variations in *Toxoplasma gondii* strains influence the severity and recurrence rate of ocular toxoplasmosis in different geographic regions?', 'What are the diagnostic efficacy and clinical limitations of using PCR analysis of aqueous humor compared to serological testing for confirming active ocular toxoplasmosis?']
🔎 Searching the web...
   - Searching: What are the current international consensus guidelines regarding the optimal duration and combination of antimicrobial therapy for active ocular toxoplasmosis?
   - Searching: How do g

# Comprehensive Research Summary: Ocular Toxoplasmosis (OT)

Ocular toxoplasmosis (OT), caused by the intracellular parasite *Toxoplasma gondii* (Tg), remains the leading infectious cause of posterior uveitis worldwide. It is classified as a neglected tropical disease, characterized by significant variability in clinical manifestation, severity, and recurrence rates across different geographic regions. Recent international consensus efforts aim to standardize diagnostic and therapeutic approaches, particularly concerning optimal antimicrobial regimens and the influence of parasite genotyping.

---

## I. Diagnosis and Confirmation of Active Ocular Toxoplasmosis

While clinical presentation and serological testing (IgG/IgM) remain foundational, laboratory confirmation is often necessary, especially in atypical or immunocompromised cases.

### Role of Polymerase Chain Reaction (PCR)

Current findings support the use of PCR analysis of the aqueous humor as a robust diagnostic tool.

*   **Efficacy:** PCR demonstrates independent efficacy when paired with serum samples, offering a highly specific method for confirming an active infection within the eye.
*   **Clinical Utility:** A primary benefit of aqueous humor PCR is its ability to effectively discriminate Ocular Toxoplasmosis from other ocular diseases that may present similarly (e.g., viral retinitis), thereby avoiding the diagnostic ambiguities associated with relying solely on systemic serology.
*   **Atypical Cases:** Consensus guidelines support the selective use of PCR testing for atypical lesions or cases where the diagnosis is uncertain.

---

## II. International Consensus on Antimicrobial Therapy

The management of active OT typically requires a combination of antiparasitic agents and adjunctive corticosteroids, with treatment decisions highly individualized based on the ophthalmologic evaluation.

### A. Standard Therapeutic Regimen and Duration

Current international consensus guidelines support the use of antiparasitic therapy in most active cases, primarily utilizing a triple-drug combination:

1.  **Pyrimethamine**
2.  **Sulfadiazine**
3.  **Folinic Acid (Leucovorin):** Essential to mitigate the hematologic toxicity (e.g., myelosuppression) associated with Pyrimethamine.

For example, a standard pediatric dose involves Pyrimethamine (2 mg/kg loading dose, then 1 mg/kg/day), plus Sulfadiazine (50 mg/kg two times per day), plus Folinic Acid (7.5 mg per day).

**Optimal Duration:**
Therapy is generally recommended for a duration of **4 to 6 weeks**, followed by a comprehensive reevaluation of the patient's condition.

### B. Adjunctive Corticosteroid Use

Corticosteroids are frequently prescribed in addition to antiparasitic agents. Their primary role is to control the intense intraocular inflammation caused by the parasite and the host response. However, corticosteroids must **never** be initiated without concurrent antiparasitic coverage, as they can exacerbate the infection.

### C. Clinical Decision-Making Parameters

The decision to initiate treatment for OT is complex and hinges on several critical clinical parameters determined by a complete ophthalmologic evaluation:

*   **Acuteness and Persistence:** Treatment is reserved for active lesions; healed lesions should not be treated.
*   **Lesion Location:** Lesions involving the macula or optic nerve head are highly prioritized for treatment due to the high risk of permanent visual impairment.
*   **Visual Acuity:** The degree of vision loss often dictates the urgency of intervention.
*   **Size and Inflammation:** Large lesions or those associated with severe vitritis require aggressive management.

### D. Management in Immunocompromised Patients

Toxoplasmosis is highly dangerous in immunodeficient patients (e.g., those with AIDS) and is often fatal if left untreated.

*   **Extended Duration:** Treatment is recommended for at least **4 to 6 weeks beyond the resolution of all clinical signs and symptoms**, and often requires maintenance therapy for 6 months or longer.
*   **Maintenance Therapy:** For patients with HIV/AIDS, maintenance therapy is recommended until a significant immunological improvement is achieved, usually in response to antiretroviral therapy (ART). Relapses are common in this patient population.

---

## III. Geographic and Genotypic Influence on Disease Severity

The clinical features, severity, and recurrence rate of Ocular Toxoplasmosis are significantly influenced by interactions between host factors, environmental factors, and genotypic variations in *Toxoplasma gondii* strains.

### A. Geographic Variation in Virulence

Recurrences and severe disease are not uniformly distributed worldwide. Studies indicate that:

*   **High-Recurrence Regions:** Recurrences, visual impairment, and blindness are most common among patients in **South American and Central American latitudes**.
*   **Virulent Strains:** This regional disparity strongly suggests that these geographies harbor more virulent *Toxoplasma gondii* strains, contributing to more severe cases of OT compared to strains found in other parts of the world.

### B. Risk Factors for Recurrence

A systematic review identified multiple factors associated with a greater risk of recurrence and severe disease:

| Factor Type | Specific Risk Factors |
| :--- | :--- |
| **Parasite Factors** | More virulent strains; geographical region where the infection was acquired. |
| **Host/Clinical Factors** | Age older than 40 years; congenital toxoplasmosis; bilateral compromise; *de novo* OT lesions or recurrence within one year of the first episode. |
| **Lesion Factors** | Involvement of the macular area; lesions greater than one disc diameter in size. |
| **Environmental Factors** | High precipitation levels. |

---

## IV. Conclusion

Ocular Toxoplasmosis management is driven by international collaboration (including bodies like TITAN and APSOII) seeking consensus on therapeutic best practices. While the triple-drug therapy remains the standard for active lesions, clinical decisions must prioritize location and severity. Crucially, the recognition that genotypic variations in *Toxoplasma gondii* strains profoundly affect recurrence rates highlights the need for regionally tailored strategies, particularly in high-virulence zones like South America. Advancements in molecular diagnostics, such as aqueous humor PCR, further aid in differentiating OT from imitators, improving therapeutic specificity.